# 1. Package 설치

In [ ]:
%pip install langchain langchain-core langchain-community langchain-text-splitters langchain-openai langchain-pinecone

# 2. Knowledge Base 구성을 위한 데이터 생성

- 4. LangChain을 활용한 Vector Database 변경 (Chroma -> Pinecone)과 동일함

In [10]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

# 민법
# document_name = './civil_with_markdown.docx'
# 형법
document_name = './criminal_with_markdown.docx'

loader = Docx2txtLoader(document_name)
document_list = loader.load_and_split(text_splitter=text_splitter)

In [11]:
len_list = len(document_list)
len_list

43

In [12]:
document_list[len_list - 10]

Document(metadata={'source': './criminal_with_markdown.docx'}, page_content='② 제287조부터 제289조까지의 죄를 범하여 약취, 유인, 매매 또는 이송된 사람을 상해에 이르게 한 때에는 2년 이상 20년 이하의 징역에 처한다.\n\n[전문개정 2013. 4. 5.]\n\n\n\n제291조(약취, 유인, 매매, 이송 등 살인ㆍ치사) ① 제287조부터 제289조까지의 죄를 범하여 약취, 유인, 매매 또는 이송된 사람을 살해한 때에는 사형, 무기 또는 7년 이상의 징역에 처한다.\n\n② 제287조부터 제289조까지의 죄를 범하여 약취, 유인, 매매 또는 이송된 사람을 사망에 이르게 한 때에는 무기 또는 5년 이상의 징역에 처한다.\n\n[전문개정 2013. 4. 5.]\n\n\n\n제292조(약취, 유인, 매매, 이송된 사람의 수수ㆍ은닉 등) ① 제287조부터 제289조까지의 죄로 약취, 유인, 매매 또는 이송된 사람을 수수(授受) 또는 은닉한 사람은 7년 이하의 징역에 처한다.\n\n② 제287조부터 제289조까지의 죄를 범할 목적으로 사람을 모집, 운송, 전달한 사람도 제1항과 동일한 형으로 처벌한다.\n\n[전문개정 2013. 4. 5.]\n\n\n\n제293조 삭제 <2013. 4. 5.>\n\n\n\n제294조(미수범) 제287조부터 제289조까지, 제290조제1항, 제291조제1항과 제292조제1항의 미수범은 처벌한다.\n\n[전문개정 2013. 4. 5.]\n\n\n\n제295조(벌금의 병과) 제288조부터 제291조까지, 제292조제1항의 죄와 그 미수범에 대하여는 5천만원 이하의 벌금을 병과할 수 있다.\n\n[전문개정 2013. 4. 5.]\n\n\n\n제295조의2(형의 감경) 제287조부터 제290조까지, 제292조와 제294조의 죄를 범한 사람이 약취, 유인, 매매 또는 이송된 사람을 안전한 장소로 풀어준 때에는 그 형을 감경할 수 있다.\n\n[전문개정 2013. 4. 5.]\

In [13]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [14]:
import os

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

is_create = False
# 민법
# index_name = 'civil-markdown-index'
# 형법
index_name = 'criminal-markdown-index'

pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

if is_create:
    # 데이터를 추가할 때
    database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)
else:
    # 이미 생성된 데이터베이스를 사용할 때 
    database = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embedding)

In [15]:
# 민법
# query = '부동산 매매 계약에서의 소유 이전은 어떻게 이루어지나요?'
# 형법
query = '소유권 이전 과정에서 허위 서류를 제출한 경우, 이는 어떤 형법적 처벌을 받을 수 있나요?'

# 3. 답변 생성을 위한 Retrieval

- RetrievalQA에 전달하기 위해 retriever 생성
- search_kwargs의 k값을 변경해서 가져올 문서의 갯수를 지정할 수 있음
- .invoke()를 호출해서 어떤 문서를 가져오는지 확인 가능

In [16]:
retriever = database.as_retriever(search_kwargs={'k': 4})
retriever.invoke(query)

[Document(metadata={'source': './criminal_with_markdown.docx'}, page_content='[전문개정 1995. 12. 29.]\n\n\n\n제222조(인지ㆍ우표유사물의 제조 등) ①판매할 목적으로 대한민국 또는 외국의 공채증서, 인지, 우표 기타 우편요금을 표시하는 증표와 유사한 물건을 제조, 수입 또는 수출한 자는 2년 이하의 징역 또는 500만원 이하의 벌금에 처한다. <개정 1995. 12. 29.>\n\n②전항의 물건을 판매한 자도 전항의 형과 같다.\n\n[제목개정 1995. 12. 29.]\n\n\n\n제223조(미수범) 제214조 내지 제219조와 전조의 미수범은 처벌한다.\n\n\n\n제224조(예비, 음모) 제214조, 제215조와 제218조제1항의 죄를 범할 목적으로 예비 또는 음모한 자는 2년 이하의 징역에 처한다.\n\n\n\n제20장 문서에 관한 죄\n\n\n\n제225조(공문서등의 위조ㆍ변조) 행사할 목적으로 공무원 또는 공무소의 문서 또는 도화를 위조 또는 변조한 자는 10년 이하의 징역에 처한다. <개정 1995. 12. 29.>\n\n\n\n제226조(자격모용에 의한 공문서 등의 작성) 행사할 목적으로 공무원 또는 공무소의 자격을 모용하여 문서 또는 도화를 작성한 자는 10년 이하의 징역에 처한다. <개정 1995. 12. 29.>\n\n\n\n제227조(허위공문서작성등) 공무원이 행사할 목적으로 그 직무에 관하여 문서 또는 도화를 허위로 작성하거나 변개한 때에는 7년 이하의 징역 또는 2천만원 이하의 벌금에 처한다.\n\n[전문개정 1995. 12. 29.]\n\n\n\n제227조의2(공전자기록위작ㆍ변작) 사무처리를 그르치게 할 목적으로 공무원 또는 공무소의 전자기록등 특수매체기록을 위작 또는 변작한 자는 10년 이하의 징역에 처한다.\n\n[본조신설 1995. 12. 29.]\n\n\n\n제228조(공정증서원본 등의 부실기재) ①공무원에 대하여 허위신고를 하여 공정증서원본 또는

# 4. Augmentation을 위한 Prompt 활용

- Retrieval 된 데이터는 LangChain에서 제공하는 프롬프트("rlm/rag-prompt") 사용

In [17]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

# 5. 답변 생성

- RetrievalQA를 통해 LLM에 전달
    - RetrievalQA는 create_retrieval_chain으로 대체됨
    - 실제 ChatBot 구현 시 create_retrieval_chain으로 변경하는 과정을 볼 수 있음
- 하단의 dictionary_chain과 연계하여 사용

In [18]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [19]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

In [20]:
ai_message = qa_chain.invoke({"query": query})

In [21]:
ai_message

{'query': '소유권 이전 과정에서 허위 서류를 제출한 경우, 이는 어떤 형법적 처벌을 받을 수 있나요?',
 'result': '소유권 이전 과정에서 허위 서류를 제출한 경우, 이는 형법 제228조에 따라 5년 이하의 징역 또는 1천만원 이하의 벌금에 처할 수 있습니다.'}

# 6. Retrieval을 위한 keyword 사전 활용

- Knowledge Base에서 사용되는 keyword를 활용하여 사용자 질문 수정
- LangChain Expression Language (LCEL)을 활용한 Chain 연계

In [22]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# 민법
# dictionary = [
#     "사람을 나타내는 표현 -> 개인",
#     "재산의 이전을 나타내는 표현 -> 양도",
#     "결혼을 나타내는 표현 -> 배우자 관계",
#     "법적 책임을 나타내는 표현 -> 책임",
#     "계약의 당사자를 나타내는 표현 -> 계약 당사자"
# ]
# 형법
dictionary = [
    "범죄를 나타내는 표현 -> 범죄",
    "범죄의 피해자를 나타내는 표현 -> 피해자",
    "형벌을 나타내는 표현 -> 형벌",
    "공범을 나타내는 표현 -> 공범",
    "고의성을 나타내는 표현 -> 고의",
    "과실을 나타내는 표현 -> 과실",
    "변호권을 나타내는 표현 -> 변호권",
    "구속을 나타내는 표현 -> 구속",
    "재판을 나타내는 표현 -> 재판",
    "형의 집행을 나타내는 표현 -> 형 집행",
    "보호관찰을 나타내는 표현 -> 보호관찰",
    "자백을 나타내는 표현 -> 자백",
    "증거를 나타내는 표현 -> 증거",
    "형사 고소를 나타내는 표현 -> 고소",
    "형사 기소를 나타내는 표현 -> 기소",
    "불법 행위를 나타내는 표현 -> 불법 행위",
    "유죄를 나타내는 표현 -> 유죄",
    "무죄를 나타내는 표현 -> 무죄",
    "혐의를 나타내는 표현 -> 혐의",
    "집행유예를 나타내는 표현 -> 집행유예"
]

prompt = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    그런 경우에는 질문만 리턴해주세요.
    사전: {dictionary}

    질문: {{question}}
""")

dictionary_chain = prompt | llm | StrOutputParser()

In [23]:
new_question = dictionary_chain.invoke({"question": query})

In [24]:
new_question

'소유권 이전 과정에서 허위 서류를 제출한 경우, 이는 어떤 형벌을 받을 수 있나요?'

In [25]:
tax_chain = {"query": dictionary_chain} | qa_chain

In [26]:
ai_response = tax_chain.invoke({"question": query})

In [27]:
ai_response

{'query': '소유권 이전 과정에서 허위 서류를 제출한 경우, 이는 어떤 형벌을 받을 수 있나요?',
 'result': '소유권 이전 과정에서 허위 서류를 제출한 경우, 5년 이하의 징역 또는 1천만원 이하의 벌금에 처할 수 있습니다. 이는 제228조(공정증서원본 등의 부실기재)에 해당합니다.'}